# TP1 - REDIS - Adrián José Zapater Reig.

Tenemos:
- user_id - Que es un contador incremental automático para obtener el siguiente id de usuairo libre.
- post_id
- users - Set para convertir de id a nombre y de nombre a id. Cada usuario se da de alta 2 veces
- 

In [192]:
# Clase estática que contiene las constantes de los nombres de las colecciones de REDIS y sus campos.
class Post(object):
    # Contador incremental de id de post:
    c_post_id = 'post_id'
    
    # Hash que representa un post. {user_id, body, ts}.
    base_s = 'post:{}'
    # Campos del Hash.
    id_creador = 'user_id'
    cuerpo = 'body'
    ts = 'timestamp'

In [197]:
# Clase estática que contiene las constantes de los nombres de las colecciones de REDIS y sus campos.
class User(object):
    # Contador incremental de id de usuario:
    c_user_id = 'user_id'
    
    # Hash de traducción nombre usuario <-> id usuario. {id, username}
    # Cada usuario aparece 2 veces: 1 por el id (users:1) y otra por el nombre (users:miguel)
    base_hs = 'users:{}'
    id_usuario = 'id'
    nombre = 'username'
    
    # Set de posts creados por un id usuario o por sus followings siempre que la fecha del post sea superior a la fecha de following.

    posts_s = 'posts:{}'
    
    # Set Ordenado de los followers de un id usuario por su timestamp.
    # Habrá que expresar el TS en algún formato que permita la comparativa alfanumérica.
    followers_zs = 'followers:{}'
    
    # Set Ordenado de los followings de un id usuario por su timestamp.
    # Habrá que expresar el TS en algún formato que permita la comparativa alfanumérica.
    followings_zs = 'followings:{}'

In [338]:
class TwitterBackend(object):    
    def __init__(self, host='127.0.0.1',port=6379, password=''):
        # Usamos  decode_responses=True para evitar tener que decodificar todas las respuesdtas de REDIS.
        self._db = redis.Redis(host=host,port=port, password=password, decode_responses=True)
        
    def restarDB(self):
        '''Borra todas las keys de la base de datos.'''
        self._db.flushall()
              
    
    def nuevo_usuario(self, username):
        '''Da de alta un nuevo usuario y devuelve el nuevo id. Devuelve None si el usuario ya existía.'''
        # Comprobamos si el usuario existe.
        is_new_user = self.get_id_usuario_by_nombre(username) is None
        if(is_new_user):
            # Nuevo id de usuario.
            usr_id = self._db.incr(User.c_user_id)
            # Insertamos el nuevo usuario por id
            x = self._db.hmset(User.base_hs.format(usr_id), {User.id_usuario:usr_id, User.nombre:username})
            # Insertamos el nuevo usuario por nombre
            x = self._db.hmset(User.base_hs.format(username), {User.id_usuario:usr_id, User.nombre:username})
            return usr_id
        
    def _nuevo_follower(self, orig_usrname, follower_usrname, ts):
        '''
        Inserta un nuevo follower.
        Devuelve True si logra insertarlo y False si falla o si ya existe.
        Si no existe el usuario original o el usuario follower, también devuelve False.
        '''
        res = False
        # Obtenemos los ids del usuario original y su follower.
        id_follower = self.get_id_usuario_by_nombre(follower_usrname)
        id_orig_usr = self.get_id_usuario_by_nombre(orig_usrname)
        
        # Comprobamos si existen ambos usuarios.
        if((not id_follower is None) and (not id_orig_usr is None)):
            # Damos de alta la relación.
            # Devuelve 1 si hace el insert y 0 si no. bool(1) = True y bool(0) = False.
            res = bool(self._db.zadd(User.followers_zs.format(id_orig_usr), {id_follower:ts}))
        
        return res
    
    def _nuevo_following(self, orig_usrname, following_usrname, ts):
        '''
        Inserta un nuevo following.
        Devuelve True si logra insertarlo y False si falla o si ya existe.
        Si no existe el usuario original o el usuario follower, también devuelve False.
        '''
        res = False
        # Obtenemos los ids del usuario original y su following.
        id_following = self.get_id_usuario_by_nombre(following_usrname)
        id_orig_usr = self.get_id_usuario_by_nombre(orig_usrname)
        
        # Comprobamos si existen ambos usuarios.
        if((not id_following is None) and (not id_orig_usr is None)):
            # Damos de alta la relación.
            res = bool(self._db.zadd(User.followings_zs.format(id_orig_usr), {id_following:ts}))
        
        return res
    
    def seguir(self, orig_usrname, following_usrname, ts):
        '''
        Formaliza que un usuario original sigue a un usuario following dando de alta los following y los followers.
        Devuelve True si logra insertarlo y False en caso contrario.
        Si hay un error al dar de alta el following, no continua con el follower.
        '''
        # Validamos si TS es numerico.
        self._validaTs(ts);
        
        res = self._nuevo_following(orig_usrname, following_usrname, float(ts))
        return res and self._nuevo_follower(following_usrname, orig_usrname, float(ts))
    
    def nuevo_post(self, usrname, body, ts):
        
        # Validamos si TS es numerico.
        self._validaTs(ts);
        
        res = False
        # Recuperamos el id de usuario que escribió el tweet.
        id_usr = self.get_id_usuario_by_nombre(usrname)
        
        # Comprobamos si existe.
        if (not id_usr is None):
            # Calculamos el siguiente id.
            id_post = self._db.incr(Post.c_post_id)
            # guardamos el post.
            res = self._db.hmset(Post.base_s.format(id_post), {Post.id_creador: id_usr, Post.cuerpo:body, Post.ts:ts})
            
        # Debemos actualizar los posts por usuario.
        # Primero el autor del post:
        res = res and self._db.sadd(User.posts_s.format(id_usr), id_post)
        
        # Ahora hay que actualizar la lista de posts de los followers del autor, 
        # pero sólo si la fecha del post es posterior a la fecha del follow.
        # Obtenemos los followers del creador del post que le siguieron antes de la fecha de creación del post.
        followers = self._db.zrangebyscore(User.followers_zs.format(id_usr), 0, ts)
        for follower_id in followers:
            res = res and self._db.sadd(User.posts_s.format(follower_id), id_post)
        
        return res
        
           
    #######################
    # Auxiliary functions:#
    #######################
    def get_id_usuario_by_nombre(self, username):
        '''Devuelve el id de usuario asignado al nombre proprocionado o None si no existe.'''
        return self._db.hget(User.base_hs.format(username), User.id_usuario)
    
    def get_nombre_usuario_by_id(self, usr_id):
        '''Devuelve el nombre de usuario asignado al id proprocionado o None si no existe.'''
        return self._db.hget(User.base_hs.format(usr_id), User.nombre)
    
    def _get_tm_user(self, id_usr):
        return self._db.smembers(User.posts_s.format(id_usr))
    
    def _validaTs(self, ts):
        # ts tiene que ser un entero, sino Redis levantará un error.
        # Vamos a comprobarlo y sino levantaremos una excepción que sea facil de identificar.
        try:
            float(ts)
        except Exception as err:
            raise Exception("Timestamp tiene que ser un valor numérico, sino REDIS levantará un error.", err)
        
back = TwitterBackend()

In [357]:
back.restarDB()
back.nuevo_usuario("Adrian")
back.nuevo_usuario("Maria")
back.nuevo_usuario("Dani")
back.seguir("Maria", "Adrian", 10)
back.seguir("Dani", "Adrian", 20)

<ipython-input-338-55c6b92d8ac8>:19: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  x = self._db.hmset(User.base_hs.format(usr_id), {User.id_usuario:usr_id, User.nombre:username})
<ipython-input-338-55c6b92d8ac8>:21: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  x = self._db.hmset(User.base_hs.format(username), {User.id_usuario:usr_id, User.nombre:username})


True

In [358]:
back.nuevo_post("Adrian", "Hola Mundo!", 9)
back.nuevo_post("Adrian", "Hola Mundo!", 12)
back.nuevo_post("Adrian", "Hola Mundo!", 15)

<ipython-input-338-55c6b92d8ac8>:87: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  res = self._db.hmset(Post.base_s.format(id_post), {Post.id_creador: id_usr, Post.cuerpo:body, Post.ts:ts})


1

In [361]:
back._get_tm_user(3)

set()